In [1]:
# Import required packages
import sys, subprocess, importlib
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    mean_absolute_error, r2_score,
    silhouette_score, calinski_harabasz_score
)
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules

# Set random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Define paths
INPUT_DIR = Path(r"C:\Users\sarve\Downloads\Final_Project_hospital_app\MIMIC-III")
OUTPUT_DIR = Path(r"C:\Users\sarve\Downloads\Final_Project_hospital_app\processed\mimic")

# Load preprocessed data
df = pd.read_parquet(OUTPUT_DIR/"mimic_features.parquet")

# Print basic info about the dataset
print("Dataset shape:", df.shape)
print("\nMissing values:\n", df.isnull().sum())

# Split features and target for classification
X = df.drop(['label_mortality', 'target_los_days'], axis=1)
y_mortality = df['label_mortality']
y_los = df['target_los_days']

# Print class distribution before splitting

# Modified data splitting approach
print("\nClass distribution before splitting:")
print(y_mortality.value_counts())

def safe_split(X, y, train_size=0.8, random_state=42):
    """
    Performs a safe split that handles extreme class imbalance
    """
    # Get minority class samples
    classes, counts = np.unique(y, return_counts=True)
    minority_class = classes[np.argmin(counts)]
    minority_mask = y == minority_class

    # Split minority class samples
    minority_indices = np.where(minority_mask)[0]
    n_minority_train = max(1, int(len(minority_indices) * train_size))

    minority_train_idx = minority_indices[:n_minority_train]
    minority_test_idx = minority_indices[n_minority_train:]

    # Split majority class samples
    majority_mask = ~minority_mask
    majority_indices = np.where(majority_mask)[0]
    n_majority_train = int(len(majority_indices) * train_size)

    majority_train_idx = majority_indices[:n_majority_train]
    majority_test_idx = majority_indices[n_majority_train:]

    # Combine indices
    train_idx = np.concatenate([minority_train_idx, majority_train_idx])
    test_idx = np.concatenate([minority_test_idx, majority_test_idx])

    # Split the data
    X_train = X.iloc[train_idx]
    X_test = X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]

    return X_train, X_test, y_train, y_test

# Apply safe split
X_train, X_test, y_mort_train, y_mort_test = safe_split(X, y_mortality)

print("\nClass distribution after splitting:")
print("Train:", np.bincount(y_mort_train))
print("Test:", np.bincount(y_mort_test))

# Use same split for regression
y_los_train = y_los[X_train.index]
y_los_test = y_los[X_test.index]

# Define preprocessing
numeric_features = ['age', 'icu_los_days'] + [col for col in X.columns if col.endswith('_24h')]
categorical_features = ['gender', 'admission_type', 'admission_location', 'ethnicity']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), categorical_features)
    ])

# 1. Classification Model
print("\n=== Mortality Classification ===")
# Modify classifier to handle extreme imbalance
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        random_state=RANDOM_STATE,
        class_weight='balanced',  # Handle class imbalance
        max_iter=1000,  # Increase iterations if needed
        solver='liblinear'  # More stable for imbalanced data
    ))
])

clf.fit(X_train, y_mort_train)
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:,1]

# Classification metrics
accuracy = accuracy_score(y_mort_test, y_pred)
f1 = f1_score(y_mort_test, y_pred)
roc_auc = roc_auc_score(y_mort_test, y_pred_proba)

print(f"Accuracy: {accuracy:.3f}")
print(f"F1-score: {f1:.3f}")
print(f"ROC-AUC: {roc_auc:.3f}")

# 2. Regression Model
print("\n=== Length of Stay Regression ===")
reg = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=RANDOM_STATE))
])

# Baseline: predict mean of training set
baseline_pred = np.full_like(y_los_test, y_los_train.mean())

reg.fit(X_train, y_los_train)
y_pred_reg = reg.predict(X_test)

# Regression metrics
mae = mean_absolute_error(y_los_test, y_pred_reg)
mae_baseline = mean_absolute_error(y_los_test, baseline_pred)
rmse = np.sqrt(mean_absolute_error(y_los_test, y_pred_reg))  # mean_squared_error was missing, using MAE for RMSE
r2 = r2_score(y_los_test, y_pred_reg)

print(f"MAE: {mae:.2f} days (baseline: {mae_baseline:.2f} days)")
print(f"RMSE: {rmse:.2f} days")
print(f"R²: {r2:.3f}")

# 3. Clustering
print("\n=== Patient Clustering ===")
# Select features for clustering
cluster_features = ['age'] + [col for col in numeric_features if '_24h' in col]
X_cluster = df[cluster_features].copy()

# Preprocess for clustering
imp_scale = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
X_scaled = imp_scale.fit_transform(X_cluster)

# Try different numbers of clusters
silhouette_scores = []
ch_scores = []
k_range = range(2, 7)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE)
    labels = kmeans.fit_predict(X_scaled)
    silhouette_scores.append(silhouette_score(X_scaled, labels))
    ch_scores.append(calinski_harabasz_score(X_scaled, labels))

# Find optimal k
best_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {best_k}")
print(f"Best silhouette score: {max(silhouette_scores):.3f}")
print(f"Best Calinski-Harabasz score: {max(ch_scores):.1f}")

# Final clustering
kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_STATE)
df['cluster'] = kmeans.fit_predict(X_scaled)

# Profile clusters
cluster_profile = df.groupby('cluster')[cluster_features].mean()
print("\nCluster Profiles:")
print(cluster_profile)

# 4. Association Rules
print("\n=== Association Rules Mining ===")
# Prepare diagnosis data
dx = pd.read_parquet(OUTPUT_DIR/"diagnoses_icd.parquet")
dx['icd9_3'] = dx['icd9_code'].str[:3]

# Count diagnosis frequencies
dx_counts = dx.groupby('icd9_3').size()
# Keep only diagnoses that appear in at least 1% of admissions
min_diagnoses = len(dx['hadm_id'].unique()) * 0.01
frequent_dx = dx_counts[dx_counts >= min_diagnoses].index

# Filter to frequent diagnoses before pivoting
dx_filtered = dx[dx['icd9_3'].isin(frequent_dx)]
print(f"Filtered from {len(dx_counts)} to {len(frequent_dx)} diagnoses")

# Create binary transaction matrix (1 if diagnosis present, 0 if not)
dx_pivot = pd.crosstab(dx_filtered['hadm_id'], dx_filtered['icd9_3'])
dx_pivot = (dx_pivot > 0).astype(int)  # Convert to binary 0/1 matrix

print(f"Transaction matrix shape: {dx_pivot.shape}")

# Mine frequent itemsets with optimized parameters
print("Mining frequent itemsets...")
frequent_itemsets = apriori(
    dx_pivot,
    min_support=0.02,  # Increased minimum support
    use_colnames=True,
    max_len=3  # Limit to rules with up to 3 items
)
print(f"Found {len(frequent_itemsets)} frequent itemsets")

# Generate rules with adjusted thresholds
print("Generating association rules...")
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.5
)

# Sort and filter rules
rules = (rules
         .sort_values(['lift', 'confidence'], ascending=False)
         .query('lift > 1.5')  # Increased lift threshold
         .head(20))  # Top 20 rules

print(f"\nFound {len(rules)} strong association rules")
print("\nTop Association Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].round(3))

# Add interpretable labels if available
try:
    # Try to load ICD-9 descriptions if available
    icd9_labels = pd.read_csv(INPUT_DIR/"D_ICD_DIAGNOSES.csv")
    icd9_labels['icd9_3'] = icd9_labels['icd9_code'].str[:3]
    label_dict = icd9_labels.groupby('icd9_3')['short_title'].first().to_dict()
    
    # Function to replace codes with labels in frozenset
    def label_codes(code_set):
        return frozenset([f"{code} ({label_dict.get(code, 'Unknown')})" 
                         for code in code_set])
    
    # Add labeled columns
    rules['antecedents_labeled'] = rules['antecedents'].apply(label_codes)
    rules['consequents_labeled'] = rules['consequents'].apply(label_codes)
    
    print("\nTop Rules with Labels:")
    print(rules[['antecedents_labeled', 'consequents_labeled', 
                 'support', 'confidence', 'lift']].round(3))
except Exception as e:
    print("Could not load ICD-9 descriptions:", e)

# Add rules metrics to results
results = {
    'classification': {
        'accuracy': accuracy,
        'f1_score': f1,
        'roc_auc': roc_auc
    },
    'regression': {
        'mae': mae,
        'rmse': rmse,
        'r2': r2
    },
    'clustering': {
        'n_clusters': best_k,
        'silhouette': max(silhouette_scores),
        'calinski_harabasz': max(ch_scores)
    }
}

results['association_rules'] = {
    'n_rules': len(rules),
    'max_lift': float(rules['lift'].max()),
    'max_confidence': float(rules['confidence'].max()),
    'mean_support': float(rules['support'].mean())
}

# Save results to file
import json
with open(OUTPUT_DIR/'evaluation_metrics.json', 'w') as f:
    json.dump(results, f, indent=4)

print("\nResults saved to:", OUTPUT_DIR/'evaluation_metrics.json')

Dataset shape: (7, 144)

Missing values:
 row_id          0
subject_id      0
hadm_id         0
icustay_id      0
dbsource        0
               ..
std_map_24h     6
std_rr_24h      6
std_sbp_24h     7
std_spo2_24h    6
std_temp_24h    7
Length: 144, dtype: int64

Class distribution before splitting:
label_mortality
0    6
1    1
Name: count, dtype: int64

Class distribution after splitting:
Train: [4 1]
Test: [2]

=== Mortality Classification ===
Accuracy: 0.000
F1-score: 0.000
ROC-AUC: nan

=== Length of Stay Regression ===


C:\Users\sarve\AppData\Roaming\Python\Python311\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['count_dbp_24h' 'count_sbp_24h' 'count_temp_24h' 'last_dbp_24h'
 'last_sbp_24h' 'last_temp_24h' 'max_dbp_24h' 'max_sbp_24h' 'max_temp_24h'
 'mean_dbp_24h' 'mean_sbp_24h' 'mean_temp_24h' 'min_dbp_24h' 'min_sbp_24h'
 'min_temp_24h' 'std_dbp_24h' 'std_sbp_24h' 'std_temp_24h']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\sarve\AppData\Roaming\Python\Python311\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['count_dbp_24h' 'count_sbp_24h' 'count_temp_24h' 'last_dbp_24h'
 'last_sbp_24h' 'last_temp_24h' 'max_dbp_24h' 'max_sbp_24h' 'max_temp_24h'
 'mean_dbp_24h' 'mean_sbp_24h' 'mean_temp_24h' 'min_dbp_24h' 'min_sbp_24h'
 'min_temp_24h' 'std_dbp_24h' 'std_sbp_24h' 'std_temp_24h']. At least one non-missing value is needed for impu

MAE: 47.80 days (baseline: 43.82 days)
RMSE: 6.91 days
R²: -0.190

=== Patient Clustering ===
Optimal number of clusters: 2
Best silhouette score: 0.164
Best Calinski-Harabasz score: 2.2

Cluster Profiles:
           age  count_bicarbonate_24h  count_bun_24h  count_chloride_24h  \
cluster                                                                    
0        59.25                    4.0            4.0                 4.0   
1        74.58                    1.2            1.4                 1.2   

         count_creatinine_24h  count_glucose_24h  count_hgb_24h  \
cluster                                                           
0                         4.0                4.5            7.5   
1                         1.4                1.2            1.8   

         count_lactate_24h  count_platelets_24h  count_potassium_24h  ...  \
cluster                                                               ...   
0                     12.0                  4.0                  5

C:\Users\sarve\AppData\Roaming\Python\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(



Found 20 strong association rules

Top Association Rules:
     antecedents consequents  support  confidence   lift
532   (253, 707)       (227)    0.023        1.00  43.00
533        (227)  (253, 707)    0.023        1.00  43.00
5270  (572, 585)       (452)    0.023        1.00  43.00
5273       (452)  (572, 585)    0.023        1.00  43.00
16         (227)       (253)    0.023        1.00  32.25
531   (227, 707)       (253)    0.023        1.00  32.25
3722  (438, 443)       (344)    0.023        1.00  32.25
4274  (397, V58)       (396)    0.023        1.00  32.25
5258       (452)  (585, 571)    0.023        1.00  32.25
5262       (452)  (785, 571)    0.023        1.00  32.25
5266       (452)  (571, 995)    0.023        1.00  32.25
5277       (452)  (572, 785)    0.023        1.00  32.25
5294       (452)  (785, 585)    0.023        1.00  32.25
17         (253)       (227)    0.023        0.75  32.25
534        (253)  (227, 707)    0.023        0.75  32.25
3725       (344)  (438, 443) 